**HECTOR** ___________Playing around **DSPy**


**What is DSPy?**


DSPy is a framework designed to make working with language models (like GPT-4) easier and more modular. Instead of manually writing complex prompts for each task, DSPy allows you to write Python code to interact with your models. It focuses on declarative programming, meaning you describe what you want, and the system figures out how to achieve it.


**Install Required Libraries**

In [1]:
!pip install openai PyPDF2 dspy


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 383.6/383.6 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 231.8/231.8 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.6/177.6 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 M

**Upload Your PDF File**

In [2]:
from google.colab import files

# Upload your proposal PDF
uploaded = files.upload()

# Get the filename (assumes a single file is uploaded)
pdf_filename = list(uploaded.keys())[0]
print("Uploaded file:", pdf_filename)


Saving sample-research-proposal.pdf to sample-research-proposal.pdf
Uploaded file: sample-research-proposal.pdf


**Extract Text from the PDF**

In [3]:
import PyPDF2

def extract_text_from_pdf(pdf_path):
    """Extract text from a PDF file using PyPDF2."""
    with open(pdf_path, "rb") as f:
        reader = PyPDF2.PdfReader(f)
        text = ""
        for page in reader.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text

# Extract text from the uploaded PDF
pdf_text = extract_text_from_pdf(pdf_filename)
print("Extracted text length:", len(pdf_text))


Extracted text length: 22449


**Configure DSPy and OpenAI API**

In [4]:
import openai
import dspy

# Set your OpenAI API key here
openai.api_key = "YOUR_OPENAI_API_KEY"

# Configure DSPy with your language model (e.g., GPT‑4)
lm = dspy.LM("openai/gpt-4", api_key=openai.api_key)
dspy.configure(lm=lm)


In [5]:
# Define a custom signature for summarization
class SummarizeSignature(dspy.Signature):
    """
    Given a passage (e.g. a proposal document), generate a concise summary.
    """
    passage = dspy.InputField(desc="The document text to be summarized.")
    summary: str = dspy.OutputField(desc="A clear and concise summary of the document.")

# Build a summarization module using DSPy's ChainOfThought
summarizer = dspy.ChainOfThought(SummarizeSignature)

# Define a custom prompt to guide the summarization for proposal PDFs
custom_prompt = (
    "You are an expert summarizer for business proposals. "
    "Please summarize the following document, highlighting the key objectives, "
    "methodology, deliverables, budget, and timelines in a clear and concise manner."
)



**Split PDF Text into Chunks**


In [6]:
def split_text(text, max_chunk_size=2000):
    """
    Split text into chunks of at most max_chunk_size characters,
    preserving whole words.
    """
    words = text.split()
    chunks = []
    current_chunk = []
    current_length = 0
    for word in words:
        # +1 accounts for space between words
        if current_length + len(word) + 1 <= max_chunk_size:
            current_chunk.append(word)
            current_length += len(word) + 1
        else:
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]
            current_length = len(word) + 1
    if current_chunk:
        chunks.append(" ".join(current_chunk))
    return chunks

# Split the extracted PDF text into chunks
chunks = split_text(pdf_text, max_chunk_size=2000)
print("Number of chunks:", len(chunks))


Number of chunks: 11



**Summarize Each Chunk and Combine Summaries**

In [7]:
# Summarize each chunk using our summarization module
chunk_summaries = []
for i, chunk in enumerate(chunks):
    result = summarizer(passage=chunk, prompt=custom_prompt)
    summary_text = result.summary
    chunk_summaries.append(summary_text)
    print(f"Chunk {i+1} summary:")
    print(summary_text)
    print("-" * 50)

# Combine the chunk summaries into one text
combined_summary_text = "\n".join(chunk_summaries)



Chunk 1 summary:
The document introduces two research proposals from the Department of Social Policy and Criminology. The first proposal explores the experiences of fathers post-divorce or separation, focusing on their employment and caring responsibilities. It emphasizes the need for more research on the experiences of working-class fathers. The second proposal, not detailed in the passage, is about police governance. Both proposals serve as examples for postgraduate research.
--------------------------------------------------
Chunk 2 summary:
The proposed research aims to explore the identity and roles of fathers post-divorce or separation, focusing on their experiences, perceptions, and how they negotiate employment and caring responsibilities. The study contributes to the growing field of 'family practice' sociology and could challenge traditional gender roles in earning and caring. The research is politically and sociologically significant, as it may provide insights into the real